# Solving the Line Extending Game with Rules

In [1]:
%load_ext autoreload
%autoreload 2

In [64]:
import random
import warnings
import time
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import CFG
from nltk.parse.generate import generate
from nltk.tokenize import wordpunct_tokenize
from nltk.parse.recursivedescent import RecursiveDescentParser
from nltk.tree.tree import Tree

from nsai_experiments import line_extending_game_tools as lgt

Taking inspiration from the second human solution, we define a *rule* as a statement of the form "if this list of cells is filled in and this list of cells is not filled in, fill in the central cell." Then a *ruleset* is a list of rules with the semantic "fill in the central cell if any of these rules say to do so." Here, we will develop a context-free grammar that can produce rules, then train an agent to play the game with a ruleset as its policy.

In [26]:
grammar = CFG.fromstring("""
    S -> "if_filled" CellList "and_empty" CellList "then_activate"
    CellList -> "[" Cell "]" CellList |
    Cell -> "R" NonCentralNumber "C" AnyNumber | "R" CentralNumber "C" NonCentralNumber
    AnyNumber -> NonCentralNumber | CentralNumber
    NonCentralNumber -> "0" | "1" | "3" | "4"
    CentralNumber -> "2"
    """)

Note that we have hardcoded the idea that the central cell (R 2 C 2) is invalid, but there is no elegant way (TODO pumping lemma?) to encode in the *grammar* that cells that appear in the "if_filled" list should not appear in the "and_empty" list. In other words, it is possible to have rules that are grammatical but unpragmatic.

In [28]:
some_rules = [" ".join(x) for x in generate(grammar, depth = 6)]
display(some_rules[:5])

['if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] [ R 2 C 0 ] then_activate',
 'if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] [ R 2 C 1 ] then_activate',
 'if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] [ R 2 C 3 ] then_activate',
 'if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] [ R 2 C 4 ] then_activate',
 'if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] then_activate']

This is an LL(1) language, so a recursive descent parser suffices:

In [44]:
my_rule = some_rules[1]
parser = RecursiveDescentParser(grammar)
parsed = parser.parse(wordpunct_tokenize(my_rule))
for x in parsed:
    print(x)

(S
  if_filled
  (CellList
    [
    (Cell R (NonCentralNumber 0) C (AnyNumber (NonCentralNumber 0)))
    ]
    (CellList
      [
      (Cell R (CentralNumber 2) C (NonCentralNumber 0))
      ]
      (CellList )))
  and_empty
  (CellList
    [
    (Cell R (NonCentralNumber 0) C (AnyNumber (NonCentralNumber 0)))
    ]
    (CellList
      [
      (Cell R (CentralNumber 2) C (NonCentralNumber 1))
      ]
      (CellList )))
  then_activate)


Here we parse a rule string into a Python data structure for further use:

In [106]:
def parse_cell(cell: Tree):
    _, row, _, col = cell
    # row, col at this point might be primitive or composite, but in all cases the number is at the end
    return int(row.leaves()[-1]), int(col.leaves()[-1])

def parse_cell_list(cl_tree: Tree):
    if len(cl_tree) == 0: return []
    _, first, _, rest = cl_tree
    return [parse_cell(first)] + parse_cell_list(rest)

def parse_to_python(rule_string):
    root, = parser.parse(wordpunct_tokenize(rule_string))
    _, cl_tree1, _, cl_tree2, _ = root
    return parse_cell_list(cl_tree1), parse_cell_list(cl_tree2)

display(my_rule)
display(parse_to_python(my_rule))

'if_filled [ R 0 C 0 ] [ R 2 C 0 ] and_empty [ R 0 C 0 ] [ R 2 C 1 ] then_activate'

([(0, 0), (2, 0)], [(0, 0), (2, 1)])